# Лабораторная работа №1

Требования:
* Python >= 3.X
* NLTK >= 3.2.5

Лабораторную работу необходимо выполнять в данном шаблоне. Результат работы выслать письмом на litvinov.vg@ssau.ru. В теме письма указывать ФИО полностью.

Для выполнения задания необходимо подобрать корпус текстов (художественных) на английском языке. Объем корпуса должен составлять не менее $3 \cdot 10^7$ символов. Далее все действия будут выполнятся над выбранными данными. Для отладки алгоритмов лучше использовать не весь корпус, а лишь его часть.

In [ ]:
import nltk


nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
with open('/content/Data.txt', 'r', encoding='utf-8') as f:
    data = f.read()
    
len(data)

2294245

#### Шаг №1
Заменить все числа, которые представлены цифрами, их текстовым представлением (т.е. прописью). (1 = one, 23 = twenty three, 1042 = one thousand forty two, и т.п.). Методы библиотек не использовать, алгоритм необходимо написать самостоятельно.

In [ ]:
from typing import Dict, Union, Optional, Iterable, List, Match, Tuple, Callable
import re

DIGIT_GROUP = re.compile(r"(\d)")
TWO_DIGITS = re.compile(r"(\d)(\d)")
THREE_DIGITS = re.compile(r"(\d)(\d)(\d)")
THREE_DIGITS_WORD = re.compile(r"(\d)(\d)(\d)(?=\D*\Z)")
TWO_DIGITS_WORD = re.compile(r"(\d)(\d)(?=\D*\Z)")
ONE_DIGIT_WORD = re.compile(r"(\d)(?=\D*\Z)")

FOUR_DIGIT_COMMA = re.compile(r"(\d)(\d{3}(?:,|\Z))")
NON_DIGIT = re.compile(r"\D")
WHITESPACES_COMMA = re.compile(r"\s+,")
COMMA_WORD = re.compile(r", (\S+)\s+\Z")
WHITESPACES = re.compile(r"\s+")

_number_args = None
mill_count = 0


# NUMERICAL INFLECTIONS

nth = {
    0: "th",
    1: "st",
    2: "nd",
    3: "rd",
    4: "th",
    5: "th",
    6: "th",
    7: "th",
    8: "th",
    9: "th",
    11: "th",
    12: "th",
    13: "th",
}

ordinal = dict(
    ty="tieth",
    one="first",
    two="second",
    three="third",
    five="fifth",
    eight="eighth",
    nine="ninth",
    twelve="twelfth",
)

ordinal_suff = re.compile(fr"({'|'.join(ordinal)})\Z")


# NUMBERS

unit = ["", "one", "two", "three", "four", "five", "six", "seven", "eight", "nine"]
teen = [
    "ten",
    "eleven",
    "twelve",
    "thirteen",
    "fourteen",
    "fifteen",
    "sixteen",
    "seventeen",
    "eighteen",
    "nineteen",
]
ten = [
    "",
    "",
    "twenty",
    "thirty",
    "forty",
    "fifty",
    "sixty",
    "seventy",
    "eighty",
    "ninety",
]
mill = [
    " ",
    " thousand",
    " million",
    " billion",
    " trillion",
    " quadrillion",
    " quintillion",
    " sextillion",
    " septillion",
    " octillion",
    " nonillion",
    " decillion",
]


def millfn(ind: int = 0) -> str:
    if ind > len(mill) - 1:
        print3("number out of range")
        raise NumOutOfRangeError
    return mill[ind]

def unitfn(units: int, mindex: int = 0) -> str:
    return f"{unit[units]}{millfn(mindex)}"

def tenfn(tens, units, mindex=0):
    if tens != 1:
        tens_part = ten[tens]
        if tens and units:
            hyphen = "-"
        else:
            hyphen = ""
        unit_part = unit[units]
        mill_part = millfn(mindex)
        return f"{tens_part}{hyphen}{unit_part}{mill_part}"
    return f"{teen[units]}{mill[mindex]}"

def hundfn(hundreds: int, tens: int, units: int, mindex: int) -> str:
    global _number_args
    
    if hundreds:
        andword = f" {_number_args['andword']} " if tens or units else ""
        # use unit not unitfn as simpler
        return (
            f"{unit[hundreds]} hundred{andword}"
            f"{tenfn(tens, units)}{millfn(mindex)}, "
        )
    if tens or units:
        return f"{tenfn(tens, units)}{millfn(mindex)}, "
    return ""

def group1sub(mo: Match) -> str:
    global _number_args
    
    units = int(mo.group(1))
    if units == 1:
        return f" {_number_args['one']}, "
    elif units:
        return f"{unit[units]}, "
    else:
        return f" {_number_args['zero']}, "

def group1bsub(mo: Match) -> str:
    global _number_args
    units = int(mo.group(1))
    if units:
        return f"{unit[units]}, "
    else:
        return f" {_number_args['zero']}, "

def group2sub(mo: Match) -> str:
    global _number_args
    
    tens = int(mo.group(1))
    units = int(mo.group(2))
    if tens:
        return f"{tenfn(tens, units)}, "
    if units:
        return f" {_number_args['zero']} {unit[units]}, "
    return f" {_number_args['zero']} {_number_args['zero']}, "

def group3sub(mo: Match) -> str:
    global _number_args
    
    hundreds = int(mo.group(1))
    tens = int(mo.group(2))
    units = int(mo.group(3))
    if hundreds == 1:
        hunword = f" {_number_args['one']}"
    elif hundreds:
        hunword = str(unit[hundreds])
    else:
        hunword = f" {_number_args['zero']}"
    if tens:
        tenword = tenfn(tens, units)
    elif units:
        tenword = f" {_number_args['zero']} {unit[units]}"
    else:
        tenword = f" {_number_args['zero']} {_number_args['zero']}"
    return f"{hunword} {tenword}, "

def hundsub(mo: Match) -> str:
    global mill_count
    ret = hundfn(
        int(mo.group(1)), int(mo.group(2)), int(mo.group(3)), mill_count
    )
    mill_count += 1
    return ret

def tensub(mo: Match) -> str:
    global mill_count
    return f"{tenfn(int(mo.group(1)), int(mo.group(2)), mill_count)}, "

def unitsub(mo: Match) -> str:
    global mill_count
    return f"{unitfn(int(mo.group(1)), mill_count)}, "

def enword(num: str, group: int) -> str:
    global _number_args
    global mill_count

    if group == 1:
        num = DIGIT_GROUP.sub(group1sub, num)
    elif group == 2:
        num = TWO_DIGITS.sub(group2sub, num)
        num = DIGIT_GROUP.sub(group1bsub, num, 1)
    elif group == 3:
        num = THREE_DIGITS.sub(group3sub, num)
        num = TWO_DIGITS.sub(group2sub, num, 1)
        num = DIGIT_GROUP.sub(group1sub, num, 1)
    elif int(num) == 0:
        num = _number_args["zero"]
    elif int(num) == 1:
        num = _number_args["one"]
    else:
        num = num.lstrip().lstrip("0")
        mill_count = 0
        # surely there's a better way to do the next bit
        mo = THREE_DIGITS_WORD.search(num)
        while mo:
            num = THREE_DIGITS_WORD.sub(hundsub, num, 1)
            mo = THREE_DIGITS_WORD.search(num)
        num = TWO_DIGITS_WORD.sub(tensub, num, 1)
        num = ONE_DIGIT_WORD.sub(unitsub, num, 1)
    return num

def number_to_words(
        num: Union[int, str],
        wantlist: bool = False,
        group: int = 0,
        comma: str = ",",
        andword: str = "and",
        zero: str = "zero",
        one: str = "one",
        decimal: str = "point",
        threshold: Optional[int] = None,
    ) -> Union[str, List[str]]:
    
    global _number_args
    
    _number_args = {"andword": andword, "zero": zero, "one": one}
    num = str(num)

    # Handle "stylistic" conversions (up to a given threshold)...
    if threshold is not None and float(num) > threshold:
        spnum = num.split(".", 1)
        while comma:
            (spnum[0], n) = FOUR_DIGIT_COMMA.subn(r"\1,\2", spnum[0])
            if n == 0:
                break
        try:
            return f"{spnum[0]}.{spnum[1]}"
        except IndexError:
            return str(spnum[0])

    if group < 0 or group > 3:
        raise BadChunkingOptionError
    nowhite = num.lstrip()
    if nowhite[0] == "+":
        sign = "plus"
    elif nowhite[0] == "-":
        sign = "minus"
    else:
        sign = ""

    myord = num[-2:] in ("st", "nd", "rd", "th")
    if myord:
        num = num[:-2]
    finalpoint = False
    if decimal:
        if group != 0:
            chunks = num.split(".")
        else:
            chunks = num.split(".", 1)
        if chunks[-1] == "":  # remove blank string if nothing after decimal
            chunks = chunks[:-1]
            finalpoint = True  # add 'point' to end of output
    else:
        chunks = [num]

    first: Union[int, str, bool] = 1
    loopstart = 0

    if chunks[0] == "":
        first = 0
        if len(chunks) > 1:
            loopstart = 1

    for i in range(loopstart, len(chunks)):
        chunk = chunks[i]
        # remove all non numeric \D
        chunk = NON_DIGIT.sub("", chunk)
        if chunk == "":
            chunk = "0"

        if group == 0 and (first == 0 or first == ""):
            chunk = enword(chunk, 1)
        else:
            chunk = enword(chunk, group)

        if chunk[-2:] == ", ":
            chunk = chunk[:-2]
        chunk = WHITESPACES_COMMA.sub(",", chunk)

        if group == 0 and first:
            chunk = COMMA_WORD.sub(f" {andword} \\1", chunk)
        chunk = WHITESPACES.sub(" ", chunk)
        
        chunk = chunk.strip()
        if first:
            first = ""
        chunks[i] = chunk

    numchunks = []
    if first != 0:
        numchunks = chunks[0].split(f"{comma} ")

    if myord and numchunks:
        mo = ordinal_suff.search(numchunks[-1])
        if mo:
            numchunks[-1] = ordinal_suff.sub(ordinal[mo.group(1)], numchunks[-1])
        else:
            numchunks[-1] += "th"

    for chunk in chunks[1:]:
        numchunks.append(decimal)
        numchunks.extend(chunk.split(f"{comma} "))

    if finalpoint:
        numchunks.append(decimal)

    if wantlist:
        if sign:
            numchunks = [sign] + numchunks
        return numchunks
    elif group:
        signout = f"{sign} " if sign else ""
        return f"{signout}{', '.join(numchunks)}"
    else:
        signout = f"{sign} " if sign else ""
        num = f"{signout}{numchunks.pop(0)}"
        if decimal is None:
            first = True
        else:
            first = not num.endswith(decimal)
        for nc in numchunks:
            if nc == decimal:
                num += f" {nc}"
                first = 0
            elif first:
                num += f"{comma} {nc}"
            else:
                num += f" {nc}"
        return num

In [ ]:
%%time
def replace_nubmers(data):
    def rep_num(nums, data):
        for num in nums:
            text_represenation = number_to_words(num)
            data = data.replace(num, text_represenation)
            print("Numeric: {} replaced by text representation: {}".format(num, text_represenation))
        return data
            
    st = set(re.findall(r'\d+st', data))
    nd = set(re.findall(r'\d+nd', data))
    rd = set(re.findall(r'\d+rd', data))
    th = set(re.findall(r'\d+th', data))
    default_nums = set(re.findall(r'\d+', data))
    
    data = rep_num(st, data)
    data = rep_num(nd, data)
    data = rep_num(rd, data)
    data = rep_num(th, data)
    data = rep_num(default_nums, data)
    
    return data

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 8.11 µs


In [ ]:
data = replace_nubmers(data)

Numeric: 87 replaced by text representation: eighty-seven
Numeric: 10 replaced by text representation: ten
Numeric: 100 replaced by text representation: one hundred
Numeric: 15 replaced by text representation: fifteen
Numeric: 52 replaced by text representation: fifty-two
Numeric: 74 replaced by text representation: seventy-four
Numeric: 17 replaced by text representation: seventeen
Numeric: 54 replaced by text representation: fifty-four
Numeric: 25 replaced by text representation: twenty-five
Numeric: 29 replaced by text representation: twenty-nine
Numeric: 53 replaced by text representation: fifty-three
Numeric: 56 replaced by text representation: fifty-six
Numeric: 45 replaced by text representation: forty-five
Numeric: 92 replaced by text representation: ninety-two
Numeric: 81 replaced by text representation: eighty-one
Numeric: 67 replaced by text representation: sixty-seven
Numeric: 28 replaced by text representation: twenty-eight
Numeric: 21 replaced by text representation: twen

#### Шаг №2
Преобразовать текст к нижнему или верхнему регистру.

In [ ]:
data = data.lower()

#### Шаг №3
Произвести токенезацию (токенами выступают слова), не учитывая знаки препинания (т.е. необходимо разбить текст на токены (лексемы)):

In [ ]:
def tokenize_corpus(data, tokenizer, **kwardgs):
    return tokenizer(data, **kwardgs)

* С помощью nltk.tokenize.RegexpTokenizer (тут необходимо вспомнить регулярные выражения [posix](https://ru.wikibooks.org/wiki/%D0%A0%D0%B5%D0%B3%D1%83%D0%BB%D1%8F%D1%80%D0%BD%D1%8B%D0%B5_%D0%B2%D1%8B%D1%80%D0%B0%D0%B6%D0%B5%D0%BD%D0%B8%D1%8F)).

In [ ]:
%%time
from nltk.tokenize import RegexpTokenizer

# Только буквосодержащие элементы
pattern = r'[^\W\d_]+'
tokenizer = RegexpTokenizer(pattern)

tokens = tokenize_corpus(data, tokenizer.tokenize)
print(tokens[:200])

['shakespeare', 'a', 'midsummer', 'night', 's', 'dream', 'from', 'online', 'library', 'of', 'liberty', 'unicode', 'txt', 'version', 'by', 'mike', 'scott', 'from', 'the', 'complete', 'works', 'of', 'william', 'shakespeare', 'ed', 'with', 'a', 'glossary', 'by', 'w', 'j', 'craig', 'm', 'a', 'london', 'oxford', 'university', 'press', 'onenineonesix', 'stage', 'dir', 'scene', 'athens', 'and', 'a', 'wood', 'near', 'it', 'stage', 'dir', 'act', 'one', 'scene', 'one', 'athens', 'the', 'palace', 'of', 'theseus', 'stage', 'dir', 'enter', 'theseus', 'hippolyta', 'philostrate', 'and', 'attendants', 'stage', 'dir', 'theseus', 'one', 'now', 'fair', 'hippolyta', 'our', 'nuptial', 'hour', 'draws', 'on', 'apace', 'four', 'happy', 'days', 'bring', 'in', 'another', 'moon', 'but', 'o', 'methinks', 'how', 'slow', 'this', 'old', 'moon', 'wanes', 'she', 'lingers', 'my', 'desires', 'like', 'to', 'a', 'step', 'dame', 'or', 'a', 'dowager', 'long', 'withering', 'out', 'a', 'young', 'man', 's', 'revenue', 'theseus

* С помощью nltk.word_tokenize.

In [ ]:
%%time
tokens = tokenize_corpus(data, nltk.word_tokenize)
tokens = [word for word in tokens if word.isalpha()]
print(tokens[:200])

['shakespeare', 'a', 'dream', 'from', 'online', 'library', 'of', 'liberty', 'unicode', 'version', 'by', 'mike', 'scott', 'from', 'the', 'complete', 'works', 'of', 'william', 'shakespeare', 'ed', 'with', 'a', 'glossary', 'by', 'craig', 'london', 'oxford', 'university', 'press', 'onenineonesix', 'stage', 'dir', 'and', 'a', 'wood', 'near', 'it', 'stage', 'dir', 'act', 'one', 'scene', 'one', 'athens', 'the', 'palace', 'of', 'theseus', 'stage', 'dir', 'enter', 'theseus', 'hippolyta', 'philostrate', 'and', 'attendants', 'stage', 'dir', 'theseus', 'one', 'now', 'fair', 'hippolyta', 'our', 'nuptial', 'hour', 'draws', 'on', 'apace', 'four', 'happy', 'days', 'bring', 'in', 'another', 'moon', 'but', 'o', 'methinks', 'how', 'slow', 'this', 'old', 'moon', 'wanes', 'she', 'lingers', 'my', 'desires', 'like', 'to', 'a', 'step', 'dame', 'or', 'a', 'dowager', 'long', 'withering', 'out', 'a', 'young', 'man', 'revenue', 'theseus', 'hippolyta', 'one', 'four', 'days', 'will', 'quickly', 'steep', 'themselves

#### Шаг №4
Произвести частотный анализ слов. Методы библиотек не использовать, т.е. алгоритм необходимо написать самостоятельно. Cохранить результат в выходной csv файл в порядке убывания частот. Формат выходного файла:

| Word | Count |
| --- | --- |
| cat | 1000 |
| dog | 300 |
| .... | .... |
| butterfly | 34 |

In [ ]:
import pandas as pd


def token_freq(tokens: List[str]) -> Dict:
    res = {}
    for token in tokens:
        if token in res.keys():
            res[token] += 1
        else:
            res[token] = 1
        
    return res

def word_count(tokens: List[str]) -> pd.DataFrame:
    ccount = token_freq(tokens)

    col = 'Word'
    word_sample = list(ccount.keys())[0]
    res = pd.DataFrame({col: ccount.keys(), 'Count': ccount.values()})
    res = res.sort_values('Count', ascending=False).reset_index(drop=True)
    return res

In [ ]:
res = word_count(tokens)
res.to_csv('/content/word_count.csv', index=False)
res.head()

,Word,Count
0,the,11304
1,i,10751
2,and,10256
3,to,8020
4,you,7171


#### Шаг №5
Произвести частотный анализ [N-грам](https://en.wikipedia.org/wiki/N-gram) и сохранить в выходной файл в порядке убывания частот (формат файла аналогичен предыдущему примеру, за исключением столбца Word, который заменяется на N-gram. В этой задаче можно использовать готовые библиотеки (смотрим состав библиотеки NLTK). Формат выходного файла:

| N-gram | Count |
| --- | --- |
| cat is | 1000 |
| is a | 300 |
| .... | .... |
| the fish | 34 |

* Биграм

In [ ]:
def freq_df(tokens: List[str]) -> pd.DataFrame:
    ccount = token_freq(tokens)

    col = 'N-gramm'
    word_sample = list(ccount.keys())[0]
        
    res = pd.DataFrame({col: ccount.keys(), 'Count': ccount.values()})
    res = res.sort_values('Count', ascending=False).reset_index(drop=True)
    return res

In [ ]:
from nltk import ngrams

bigramm = [' '.join(i) for i in ngrams(tokens, 2)]

res = freq_df(bigramm)
res.to_csv('/content/bigramm_count.csv', index=False)
res.head()

,N-gramm,Count
0,stage dir,5161
1,i am,928
2,i will,802
3,i have,780
4,in the,766


* 3-грам

In [ ]:
threeramm = [' '.join(i) for i in ngrams(tokens, 3)]

res = freq_df(threeramm)
res.to_csv('/content/threeramm_count.csv', index=False)
res.head()

,N-gramm,Count
0,stage dir enter,727
1,stage dir exit,450
2,stage dir exeunt,409
3,exit stage dir,294
4,dir exit stage,293


#### Шаг №6 (аналогично шагу 3)
Произвести токенезацию, но уже учитывая знаки препинания:
* С помощью nltk.tokenize.RegexpTokenizer.

In [ ]:
pattern = r'[^\W\d_]+|\.|,|!|\?'
tokenizer = RegexpTokenizer(pattern)

tokens = tokenize_corpus(data, tokenizer.tokenize)
print(tokens[:200])

['shakespeare', 'a', 'midsummer', 'night', 's', 'dream', 'from', 'online', 'library', 'of', 'liberty', 'unicode', '.', 'txt', 'version', 'by', 'mike', 'scott', 'from', 'the', 'complete', 'works', 'of', 'william', 'shakespeare', 'ed', '.', 'with', 'a', 'glossary', 'by', 'w', '.', 'j', '.', 'craig', 'm', '.', 'a', '.', 'london', 'oxford', 'university', 'press', ',', 'onenineonesix', 'stage', 'dir', 'scene', '.', 'athens', ',', 'and', 'a', 'wood', 'near', 'it', '.', 'stage', 'dir', 'act', 'one', 'scene', 'one', 'athens', '.', 'the', 'palace', 'of', 'theseus', '.', 'stage', 'dir', 'enter', 'theseus', ',', 'hippolyta', ',', 'philostrate', ',', 'and', 'attendants', '.', 'stage', 'dir', 'theseus', 'one', 'now', ',', 'fair', 'hippolyta', ',', 'our', 'nuptial', 'hour', 'draws', 'on', 'apace', 'four', 'happy', 'days', 'bring', 'in', 'another', 'moon', 'but', 'o', '!', 'methinks', 'how', 'slow', 'this', 'old', 'moon', 'wanes', 'she', 'lingers', 'my', 'desires', ',', 'like', 'to', 'a', 'step', 'da

* С помощью nltk.word_tokenize.

In [ ]:
tokens = tokenize_corpus(data, nltk.word_tokenize)
print(tokens[:200])

['shakespeare', '--', 'a', 'midsummer-night', "'s", 'dream', 'from', 'online', 'library', 'of', 'liberty', '(', 'unicode', '.txt', 'version', 'by', 'mike', 'scott', '(', 'from', '``', 'the', 'complete', 'works', 'of', 'william', 'shakespeare', "''", 'ed', '.', 'with', 'a', 'glossary', 'by', 'w.j', '.', 'craig', 'm.a', '.', '(', 'london', ':', 'oxford', 'university', 'press', ',', 'onenineonesix', ')', 'stage', 'dir', 'scene.—athens', ',', 'and', 'a', 'wood', 'near', 'it', '.', 'stage', 'dir', 'act', 'one', 'scene', 'one', 'athens', '.', 'the', 'palace', 'of', 'theseus', '.', 'stage', 'dir', 'enter', 'theseus', ',', 'hippolyta', ',', 'philostrate', ',', 'and', 'attendants', '.', 'stage', 'dir', 'theseus', 'one', '%', 'now', ',', 'fair', 'hippolyta', ',', 'our', 'nuptial', 'hour', 'draws', 'on', 'apace', ':', 'four', 'happy', 'days', 'bring', 'in', 'another', 'moon', ';', 'but', 'o', '!', 'methinks', 'how', 'slow', 'this', 'old', 'moon', 'wanes', ';', 'she', 'lingers', 'my', 'desires', '

#### Шаг №7 (аналогично шагу 4)
Произвести частотный анализ слов.

In [ ]:
res = freq_df(tokens)
res.to_csv('/content/word_count_2.csv', index=False)
res.head()

,N-gramm,Count
0,",",36845
1,.,20154
2,%,14133
3,the,11304
4,i,10751


#### Шаг №8 (аналогично шагу 5)
Произвести частотный анализ N-грам.

* Биграм

In [ ]:
from nltk import ngrams

bigramm = [' '.join(i) for i in ngrams(tokens, 2)]

res = freq_df(bigramm)
res.to_csv('/content/bigramm_count_2.csv', index=False)
res.head()

,N-gramm,Count
0,stage dir,5161
1,", and",4364
2,. stage,3278
3,", i",2178
4,% i,1440


* 3-грам

In [ ]:
threeramm = [' '.join(i) for i in ngrams(tokens, 3)]

res = freq_df(threeramm)
res.to_csv('/content/threeramm_count_2.csv', index=False)
res.head()

,N-gramm,Count
0,. stage dir,3275
1,stage dir enter,727
2,", sir ,",556
3,stage dir exit,450
4,stage dir exeunt,409
